# Getting Started With PatentsView Data Downloads

**Table of contents**<a id='toc0_'></a>    
- 1. [Overview](#toc1_)    
- 2. [Working With GUI Applications](#toc2_)    
- 3. [Working With a Command Line Interface](#toc3_)    
- 4. [Working With Python](#toc4_)    
  - 4.1. [Using Pandas](#toc4_1_)    
  - 4.2. [Faster Processing with PyDuckDB](#toc4_2_)    
- 5. [Working With R](#toc5_)    
  - 5.1. [Using Vroom](#toc5_1_)    
  - 5.2. [Using DuckPlyR](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Overview](#toc0_)

[PatentsView](https://patentsview.org) provides downloadable patent data as a set of tab-separated values (tsv) table files. This includes data on patent grants and data on pre-grant publications.

For example, on [PatentsView's Data Downloads](https://patentsview.org/download/data-download-tables) page, you can find a link to the [`g_patent`](https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip) table that contains "data on granted patents." The [data dictionary page](https://patentsview.org/download/data-download-dictionary) explains the contents of this table:

| Data Element Name | Definition                                                                 | Example                     | Years Present | Type          |
|-------------------|---------------------------------------------------------------------------|-----------------------------|---------------|---------------|
| patent_id         | patent this record corresponds to                                         | 3930271                     | all           | varchar(20)   |
| patent_type       | category of patent. Usually "design", "reissue", etc.                     | utility                     | all           | varchar(100)  |
| patent_date       | date when patent was granted                                              | 1/6/1976                    | all           | date          |
| patent_title      | title of patent                                                           | Golf glove                  | all           | mediumtext    |
| wipo_kind         | WIPO document kind codes (http://www.uspto.gov/learning-and-resources/support-centers/electronic-business-center/kind-codes-included-uspto-patent) | A                           | all           | varchar(10)   |
| num_claims        | number of claims                                                          | 4                           | all           | int(11)       |
| withdrawn         | whether a patent has been withdrawn or not (withdrawn = 1, not withdrawn = 0) | 0                           | all           | int(11)       |
| filename          | name of the raw data file where patent information is parsed from         | pftaps19760106_wk01.zip     | all           | varchar(120)  |

## 2. <a id='toc2_'></a>[Working With GUI Applications](#toc0_)

PatentsView's data downloads range in size from a few Mb to multiple Gb. Many have millions of rows. To quickly preview these files, we recommend using [Tad Viewer](https://www.tadviewer.com/), a [DuckDB](https://duckdb.org/)-based desktop application to view tabular data.

Microsoft Excel can be used, but only up to ~1M rows can be loaded at a time. Use [Microsoft PowerQuery](https://learn.microsoft.com/en-us/power-query/) to load and transform a selected subset of data into Excel for analysis.

You can also use more specialized database software such as [DBeaver](https://dbeaver.io/) or [Datagrip](https://www.jetbrains.com/datagrip/) to load and view tables using your preferred database system.

## 3. <a id='toc3_'></a>[Working With a Command Line Interface](#toc0_)

Download and unzip this table through your browser or using the commands below (using GNU's [wget](https://www.gnu.org/software/wget/)).

In [1]:
%%bash
wget --no-clobber https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip
unzip -n g_patent.tsv.zip

--2024-12-11 10:08:48--  https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.230.176, 16.15.184.139, 52.217.167.40, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.230.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220930384 (211M) [binary/octet-stream]
Saving to: ‘g_patent.tsv.zip’

     0K .......... .......... .......... .......... ..........  0% 2.10M 1m40s
    50K .......... .......... .......... .......... ..........  0% 2.28M 96s
   100K .......... .......... .......... .......... ..........  0% 84.3M 65s
   150K .......... .......... .......... .......... ..........  0% 1.92M 76s
   200K .......... .......... .......... .......... ..........  0%  100M 61s
   250K .......... .......... .......... .......... ..........  0% 69.3M 52s
   300K .......... .......... .......... .......... ..........  0%  362M 44s
   350K .......... .......... .......... .......... ..

Archive:  g_patent.tsv.zip


PatentsView uses tabs (`\t`) as a field separator, and the double-quote character (`"`) to enclose the contents of non-numeric fields. View the first two lines of `g_patent.tsv` file to validate:

In [2]:
%%bash
head -n 2 g_patent.tsv

"patent_id"	"patent_type"	"patent_date"	"patent_title"	"wipo_kind"	"num_claims"	"withdrawn"	"filename"
"10000000"	"utility"	"2018-06-19"	"Coherent LADAR using intra-pixel quadrature detection"	"B2"	20	0	"ipg180619.xml"


We recommend DuckDB as an embedded analytical database engine. It can be paired with [Halequin.sh](https://harlequin.sh) or another GUI for a user-friendly experience. Here is a read-in example:

In [3]:
%%bash 
duckdb -c "SELECT * FROM read_csv('g_patent.tsv', delim='\t', all_varchar=true) LIMIT 5;"

┌───────────┬─────────────┬─────────────┬─────────────────────────┬───────────┬────────────┬───────────┬───────────────┐
│ patent_id │ patent_type │ patent_date │      patent_title       │ wipo_kind │ num_claims │ withdrawn │   filename    │
│  varchar  │   varchar   │   varchar   │         varchar         │  varchar  │  varchar   │  varchar  │    varchar    │
├───────────┼─────────────┼─────────────┼─────────────────────────┼───────────┼────────────┼───────────┼───────────────┤
│ 10000000  │ utility     │ 2018-06-19  │ Coherent LADAR using …  │ B2        │ 20         │ 0         │ ipg180619.xml │
│ 10000001  │ utility     │ 2018-06-19  │ Injection molding mac…  │ B2        │ 12         │ 0         │ ipg180619.xml │
│ 10000002  │ utility     │ 2018-06-19  │ Method for manufactur…  │ B2        │ 9          │ 0         │ ipg180619.xml │
│ 10000003  │ utility     │ 2018-06-19  │ Method for producing …  │ B2        │ 18         │ 0         │ ipg180619.xml │
│ 10000004  │ utility     │ 2018

All data downloads are listed and documented on PatentView's website. For programmatic use, we list tables and download urls in the [`sources.yml`](sources.yml) file. For example, here is a minimal subset of tables for granted patents that we will use in examples (here using [nushell](https://www.nushell.sh/) to select from the yaml file):

In [4]:
%%bash
nu -c "open sources.yml | get granted.minimal.tables"

╭───┬──────────────────────────────╮
│ 0 │ g_patent                     │
│ 1 │ g_location_not_disambiguated │
│ 2 │ g_assignee_not_disambiguated │
│ 3 │ g_inventor_not_disambiguated │
╰───┴──────────────────────────────╯


## 4. <a id='toc4_'></a>[Working With Python](#toc0_)

### 4.1. <a id='toc4_1_'></a>[Using Pandas](#toc0_)

Using Pandas, you can load tables from disk or directly from their URL. Here is an example, loading the first five rows of the remote table:

In [5]:
import pandas as pd

# Use `dtype=str` to avoid erroneous type inference, e.g. inferring `patent_id` as a number rather than a string.
pd.read_csv(
    "https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip",
    delimiter="\t",
    dtype=str,
    nrows=5,
)

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


### 4.2. <a id='toc4_2_'></a>[Faster Processing with PyDuckDB](#toc0_)

As a faster, more memory-efficient alternative than Pandas, we recommend using [DuckDB](https://duckdb.org/). It can be used directly, through its [Python client API](https://duckdb.org/docs/api/python/overview.html), or via [Ibis](https://ibis-project.org/) for a Pandas-like experience. Here is basic usage using the Python client API:

In [6]:
import duckdb

con = duckdb.connect(database=":memory:")
data = con.read_csv("g_patent.tsv", delimiter="\t", all_varchar=True)

data.limit(5).df()

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


Use this function to programmatically extract tsv table files to a folder of your choice:

In [7]:
from zipfile import ZipFile
from urllib.request import urlretrieve
import tempfile
import os

def extract_from_url(zipfile_url: str, filename: str, dir: str='.', overwrite: bool=False):
    filepath = os.path.join(dir, filename)
    if overwrite or not os.path.exists(filepath):
        with tempfile.NamedTemporaryFile() as zipped:
            urlretrieve(zipfile_url, zipped.name)
            with ZipFile(zipped) as zipfile:
                zipfile.extract(filename)

    return filepath

extract_from_url('https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip', 'g_patent.tsv', dir='.')

'./g_patent.tsv'

You can directly read a remote zip file with DuckDB's Python client, with some utilities:

In [8]:
from contextlib import contextmanager
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile


@contextmanager
def zipfile_from_url(zipfile_url: str):
    """Open file from remote ZIP archive."""
    with urlopen(zipfile_url) as remote:
        with ZipFile(BytesIO(remote.read())) as file:
            element = file.namelist()[0]
            yield file.open(element)


def read_pv_duckdb(pv_table_url: str):
    """Read PatentsView table from URL using DuckDB."""
    with zipfile_from_url(pv_table_url) as file:
        return duckdb.read_csv(file, delimiter="\t", all_varchar=True)


table = read_pv_duckdb("https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip")
table.limit(5).df()

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


## 5. <a id='toc5_'></a>[Working With R](#toc0_)

Using R is similar to Python. First, we'll load the rpy2 extension to execute R code in this notebook.

In [9]:
%load_ext rpy2.ipython
import warnings

warnings.filterwarnings("ignore")

### 5.1. <a id='toc5_1_'></a>[Using Vroom](#toc0_)
The vroom package does not support reading remote compressed files, but it can read local zip files.

In [10]:
%%R
library(vroom)

# Vroom properly infers data types in this case.
vroom("g_patent.tsv.zip", delim = "\t")

Rows: 8980130 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (5): patent_id, patent_type, patent_title, wipo_kind, filename
dbl  (2): num_claims, withdrawn
date (1): patent_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 8,980,130 × 8
   patent_id patent_type patent_date patent_title wipo_kind num_claims withdrawn
   <chr>     <chr>       <date>      <chr>        <chr>          <dbl>     <dbl>
 1 10000000  utility     2018-06-19  Coherent LA… B2                20         0
 2 10000001  utility     2018-06-19  Injection m… B2                12         0
 3 10000002  utility     2018-06-19  Method for … B2                 9         0
 4 10000003  utility     2018-06-19  Method for … B2                18         0
 5 10000004  utility     2018-06-19  Process of … B2                 6         0
 6 10000

### 5.2. <a id='toc5_2_'></a>[Using DuckPlyR](#toc0_)

We recommend using the Tidyverse's [DuckPlyR](https://duckplyr.tidyverse.org/) as a user-friendly R client API for DuckDB.

In [11]:
%%R
library("duckplyr")

# Use "string" type to avoic errenous type inference, such as inferring `patent_id` as a number rather than a string.
duckplyr_df_from_csv("g_patent.tsv", options=list(delim="\t", all_varchar=TRUE))

duckplyr: materializing
# A tibble: 8,980,130 × 8
   patent_id patent_type patent_date patent_title wipo_kind num_claims withdrawn
   <chr>     <chr>       <chr>       <chr>        <chr>     <chr>      <chr>    
 1 10000000  utility     2018-06-19  Coherent LA… B2        20         0        
 2 10000001  utility     2018-06-19  Injection m… B2        12         0        
 3 10000002  utility     2018-06-19  Method for … B2        9          0        
 4 10000003  utility     2018-06-19  Method for … B2        18         0        
 5 10000004  utility     2018-06-19  Process of … B2        6          0        
 6 10000005  utility     2018-06-19  Article vac… B2        4          0        
 7 10000006  utility     2018-06-19  Thermoformi… B2        8          0        
 8 10000007  utility     2018-06-19  PEX expandi… B2        24         0        
 9 10000008  utility     2018-06-19  Bracelet mo… B2        11         0        
10 10000009  utility     2018-06-19  Sterile env… B2       

The duckplyr package is configured to fall back to dplyr when it encounters an
incompatibility. Fallback events can be collected and uploaded for analysis to
guide future development. By default, no data will be collected or uploaded.
→ Run `duckplyr::fallback_sitrep()` to review the current settings.
✔ Overwriting dplyr methods with duckplyr methods.
ℹ Turn off with `duckplyr::methods_restore()`.

Attachement du package : ‘duckplyr’

Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag

Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union

